In [ ]:
from statsmodels.tsa.api import VAR, AutoReg
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from statsmodels.tsa.stattools import grangercausalitytests
from tqdm import tqdm

In [ ]:
node = pd.read_csv('data/SanFrancisco_nodes.csv')
edge = pd.read_csv('data/SanFrancisco_edges.csv')
congestion = pd.read_csv('data/congestionWeek.csv')
box = [(-122.4214, 37.7907), (-122.4022, 37.7931), (-122.4013, 37.7885), (-122.4181, 37.7753)]
polygon = Polygon(box)

In [ ]:
for i in range(4468):
    if congestion[str(i)].var() <= 0.01:
        congestion = congestion.drop(str(i), axis = 1)
        continue
    if congestion[str(i)].sum() < 10:
        congestion = congestion.drop(str(i), axis = 1)
        continue

In [ ]:
df = cong
df.replace(',', '', regex=True, inplace=True) 
df = df.apply(pd.to_numeric, errors='coerce') # numeric 변환
maxlag= 1
test = 'ssr_chi2test'
cdlist=[]

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in tqdm(df.columns):
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            if min_p_value < 0.05:
                var = VAR(data[[r, c]])
                ar = AutoReg(data[[c]], 1)

                var_fit = var.fit(maxlags=1)
                ar_fit = ar.fit()
                cd = np.log(np.var(ar_fit.resid) / np.var(var_fit.resid[c]))
                temp = str(r) + ',' + str(c) + ',' + str(cd)
                cdlist.append(temp)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

da = grangers_causation_matrix(df, variables = df.columns)

In [ ]:
cd = pd.DataFrame(columns={'from', 'to', 'density'})
for st in tqdm(cdlist):
    a = st.split(',')
    cd = cd.append({'from': a[0],
                   'to': a[1],
                   'density': round(float(a[2]), 6)}, ignore_index=True)
cd.to_csv('data/single_Degree.csv')

In [ ]:
cd = pd.DataFrame(columns={'density'}, index=cong.columns)
cd=cd.fillna(0)

for st in tqdm(cdlist):
    a = st.split(',')
    cd.loc[a[0], 'density'] += float(a[2])
    
cd.to_csv('data/congestion_Degree.csv')